# MDP and gym

## Evaluation considerations
- We take into account the correctness of the solutions but also their generality and quality of the code
- Comment and discuss on the results of all your exercises (in a cell immediately after the results). You may also state the difficulties encountered, lessons learned and your understanding of the problem and solution
- Clean-up your code before submission, do not leave unnecessary code attempts, or if you deem it important, leave them in a way that it is easily understood and with comments/discussion
- We also value the originality of the solutions, don't hesitate in performing unrequested additional tasks in relation to the exercises


**NOTE**

Do not try to reproduce exactly the results in this notebook. RL training is very noisy and performances of learned policies can vary a lot, try running your trains several times.



In [ ]:
!pip install gymnasium
!pip install gymnasium[accept-rom-license,toy_text]

In [ ]:
import gymnasium as gym
from IPython.display import clear_output, HTML, display
import matplotlib.pyplot as plt
%matplotlib notebook

In [ ]:
#@title Wrapper for recording an environment into a video

from __future__ import annotations

from copy import deepcopy
from typing import Any, SupportsFloat

from gymnasium.core import ActType, ObsType, RenderFrame, WrapperActType, WrapperObsType
from gymnasium.error import DependencyNotInstalled

class RecordVideo(gym.Wrapper):
    """Adapted from https://github.com/Farama-Foundation/Gymnasium/blob/main/gymnasium/experimental/wrappers/rendering.py#L87
    """

    def __init__(self, env):
        """Initialize a :class:`HumanRendering` instance.
        Args:
            env: The environment that is being wrapped
        """
        super().__init__(env)
        assert env.render_mode in [
            "rgb_array",
            "rgb_array_list",
        ], f"Expected env.render_mode to be one of 'rgb_array' or 'rgb_array_list' but got '{env.render_mode}'"

        if "render_fps" not in env.metadata:
            env.metadata["render_fps"] = 24

        assert (
            "render_fps" in env.metadata
        ), "The base environment must specify 'render_fps' to be used with the HumanRendering wrapper"

        if "human" not in self.metadata["render_modes"]:
            self.metadata = deepcopy(self.env.metadata)
            self.metadata["render_modes"].append("human")

        self.artists = []
        self.figure = None

    @property
    def render_mode(self):
        """Always returns ``'human'``."""
        return "human"

    def step(
        self, action: WrapperActType
    ) -> tuple[WrapperObsType, SupportsFloat, bool, bool, dict]:
        """Perform a step in the base environment and render a frame to the screen."""
        result = super().step(action)
        self._render_frame()
        return result

    def reset(
        self, *, seed: int | None = None, options: dict[str, Any] | None = None
    ) -> tuple[WrapperObsType, dict[str, Any]]:
        """Reset the base environment and render a frame to the screen."""
        result = super().reset(seed=seed, options=options)
        self._render_frame()
        return result

    def video(self):
        """This method renders all frames collected up to now."""
        if self.figure is not None:
            from IPython.display import HTML
            import matplotlib.animation

            animation = matplotlib.animation.ArtistAnimation(self.figure, self.artists,
                                                             interval=1000//self.metadata["render_fps"],
                                                             blit=True,
                                                             repeat=True,
                                                             repeat_delay=2000)
            return HTML(animation.to_html5_video())

        return None

    def _render_frame(self):
        """Fetch the last frame from the base environment and render it to the screen."""
        try:
            import matplotlib.animation
            import numpy as np
        except ImportError:
            raise DependencyNotInstalled(
                "matplotlib is not installed, run `pip install matplotlib`"
            )
        if self.env.render_mode == "rgb_array_list":
            rgb_arrays = self.env.render()
        elif self.env.render_mode == "rgb_array":
            rgb_arrays = [self.env.render()]
        else:
            raise Exception(
                f"Wrapped environment must have mode 'rgb_array' or 'rgb_array_list', actual render mode: {self.env.render_mode}"
            )

        assert isinstance(rgb_arrays, list)

        for rgb_array in rgb_arrays:
            assert isinstance(rgb_array, np.ndarray)

        if self.figure is None:
            self.figure = plt.figure()
            plt.axis('off')

        self.artists.append([plt.imshow(rgb_array) for rgb_array in rgb_arrays])

    def close(self):
        """Close the rendering window."""
        result = self.video()
        super().close()

        return result

In [ ]:
import numpy as np
np.set_printoptions(linewidth=100)

Let's render soe steps to show how to use the `RecordVideo` class.

In [ ]:
env = RecordVideo(gym.make("FrozenLake-v1", render_mode="rgb_array"))
env.reset()

for _ in range(100):
    observation, reward, terminated, truncated, info = env.step(env.action_space.sample())

    done = terminated or truncated

    if done:
        env.reset()

display(env.video())

<IPython.core.display.Javascript object>

This is only needed when we want to render. In many situations of this lab we will perform many steps without the need to render, we will avoid recording a video, since doing so requires a lot of RAM (to save all the RGB frames) and we may run out of memory.

To avoid this do not wrap the environment on a `RecordVideo` class and set keyword argument `render_mode=None` when making the environment.

In [ ]:
env = gym.make("FrozenLake-v1", render_mode=None)
env.reset()

for _ in range(100):
    observation, reward, terminated, truncated, info = env.step(env.action_space.sample())

    done = terminated or truncated

    if done:
        env.reset()

In this lab we will be estimating value functions. Value functions of an MDP are conditioned on the policy used.

A policy is the probability distribution over actions. We will restrict ourselves to discrete action spaces.

We will start by using a uniform policy that chooses with equal chances between all actions.

## Exercise : A uniform policy

Create a `policy_uniform` function that returns the probability of each possible action given an environment and state.

We will also create a `sample_multinomial` function that facilitates sampling an action from the policy given the probability of each possible action.

**Hint** you may access the number of discrete actions with `env.action_space.n`.

In [ ]:
class UniformPolicy(object):
    '''
    A policy which choose an action given an action space with equal probability to every action
    '''
    def __init__(self, action_space):
        ##for the bipedal walker turned off assertion
        # assert isinstance(action_space, gym.spaces.discrete.Discrete), "Can only create uniform policies for Discrete action spaces"

        try:
            self.n_actions = action_space.n
        except:
            self.n_actions = None #for the some environments used this exception

        self.training = True

    def train(self):
        self.training = True

    def eval(self):
        self.training = False

    def probability(self, state, action):
        ### BEGIN SOLUTION
        action_prob = 1/self.n_actions #uniform probability for all
        ### END SOLUTION

        return action_prob

    def sample(self, state):
        ### BEGIN SOLUTION
        choices = [i for i in range(self.n_actions)] #have same number of choise as the action space
        action = np.random.choice(choices) #randomly choose an action
        ### END SOLUTION
        return action

# Let's instantiate a uniform policy
env = RecordVideo(gym.make("FrozenLake-v1", render_mode="rgb_array"))
uniform_policy = UniformPolicy(env.action_space)
del env
# And sample 20 actions from state==0
actions = [uniform_policy.sample(0) for i in range(20)]
print(actions)

[1, 2, 0, 2, 2, 3, 1, 2, 0, 2, 3, 2, 1, 3, 3, 0, 2, 3, 1, 3]


In [ ]:
import pprint
pprint.pprint(env.env.P)

{0: {0: [(0.3333333333333333, 0, 0.0, False),
         (0.3333333333333333, 0, 0.0, False),
         (0.3333333333333333, 4, 0.0, False)],
     1: [(0.3333333333333333, 0, 0.0, False),
         (0.3333333333333333, 4, 0.0, False),
         (0.3333333333333333, 1, 0.0, False)],
     2: [(0.3333333333333333, 4, 0.0, False),
         (0.3333333333333333, 1, 0.0, False),
         (0.3333333333333333, 0, 0.0, False)],
     3: [(0.3333333333333333, 1, 0.0, False),
         (0.3333333333333333, 0, 0.0, False),
         (0.3333333333333333, 0, 0.0, False)]},
 1: {0: [(0.3333333333333333, 1, 0.0, False),
         (0.3333333333333333, 0, 0.0, False),
         (0.3333333333333333, 5, 0.0, True)],
     1: [(0.3333333333333333, 0, 0.0, False),
         (0.3333333333333333, 5, 0.0, True),
         (0.3333333333333333, 2, 0.0, False)],
     2: [(0.3333333333333333, 5, 0.0, True),
         (0.3333333333333333, 2, 0.0, False),
         (0.3333333333333333, 1, 0.0, False)],
     3: [(0.3333333333333333,

Gym follows a formalism of MDPs that differs slightly (is more general) than the one we have seen in class. The differences are:
- In gym, transitions from a particular state and action can lead to the same destination state with different rewards (in the course all transitions reaching the same state obtain the same reward, except for transitions departing from a terminal state, whose rewards can be ignored)
- In gym, transitions to the same state can be both terminal and non-terminal (in the course it's the states that are terminal)

However we have chosen the environment `FrozenLake` which behaves as the MDPs described in the course, thus:
- All transitions to a given state have the same reward (unless the departure and destination state is the same terminal state)
- All transitions to a terminal state are terminal

In bym `env.env.P` contains the information about the process dynamics, rewards and terminal states. The information is encoded in the following way:

`env.env.P[state][action]` returns a list of tuples containing all the possible outcomes and their probability in the form `(prob, next_state, reward, terminal)` where `state` is the current state, `action` is the action taken, `probability` is the probability of the transition, `next_state` is the destination state of the transition, `reward` is the reward obtained for the transition and `terminal` is a boolean indicating if the transition ends the episode.

## Exercise : MDP from gym environment

Using this information compute the matrices $\mathcal{P}_{ss'}$, $\mathcal{R}_s$ and `terminal_states` of your MDP. Terminal is a boolean vector containing `True` for terminal states and `False` otherwise.

We will use the uniform policy that we created before, that assigns equal probability to all actions.

**Hint** all the transitions from a terminal state must be ignored.

**Optionally** while computing the matrices verify with `assert` that the conditions that we described above are fulfilled.

In [ ]:
def compute_P_and_R(env, policy):
    # Initialize the matrices P and R
    n_states = env.observation_space.n
    num_of_action = env.action_space.n
    P = np.zeros((n_states, n_states))

    R = np.empty((n_states, 1))
    R[:] = np.nan

    # Initialize a vector for terminal states
    # we will initialize with nan
    terminal_states = np.empty(n_states)

    terminal_states[:] = np.nan

    ### BEGIN SOLUTION
    # Iterate over env.env.P.items()
    # env.env.P is a dictionary that maps from
    # departure state to a dictionary of possible actions and subsequent transitions

    for i in range(n_states): #iterate through every state
        state = env.env.P[i] #take ith state
        prob_to_take_a_action = 1/num_of_action # Obtain the probability of taking that action

        # Iterate over all possible actions with equal probability
        for action in state:
            state_action_dynamics = state[action] #take every action in the state

            # Iterate over all transitions for this action
            for result in state_action_dynamics:

                next_state = result[1]
                prob = result[0]
                # If the next state is terminal flag it as such in terminal_states
                is_terminal = result[3]
                terminal_states[next_state] = is_terminal

                # Add to P the probability of the transition as:
                # (probability of the action) * (probability of the transition)

                P[i,next_state] += prob*prob_to_take_a_action

    # Convert the terminal states vector to boolean to use in indexing
    terminal_states = terminal_states.astype(bool)

    # Set the reward in R
    R = np.zeros((n_states,1))
    R[n_states-1,0] = 1 #only the last reward is 1

    return P, R, terminal_states

env = RecordVideo(gym.make("FrozenLake-v1", render_mode="rgb_array", is_slippery=False))
uniform_policy = UniformPolicy(env.action_space)
P, R, terminal_states = compute_P_and_R(env, uniform_policy)

print("Transition Probabilities (P):")
print(P)
print("\nRewards (R):")
print(R)
print("\nTerminal States:")
print(terminal_states)

Transition Probabilities (P):
[[0.5  0.25 0.   0.   0.25 0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.  ]
 [0.25 0.25 0.25 0.   0.   0.25 0.   0.   0.   0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.25 0.25 0.25 0.   0.   0.25 0.   0.   0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.25 0.5  0.   0.   0.   0.25 0.   0.   0.   0.   0.   0.   0.   0.  ]
 [0.25 0.   0.   0.   0.25 0.25 0.   0.   0.25 0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.   0.   1.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.25 0.   0.   0.25 0.   0.25 0.   0.   0.25 0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.   0.   0.   0.   1.   0.   0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.   0.25 0.   0.   0.   0.25 0.25 0.   0.   0.25 0.   0.   0.  ]
 [0.   0.   0.   0.   0.   0.25 0.   0.   0.25 0.   0.25 0.   0.   0.25 0.   0.  ]
 [0.   0.   0.   0.   0.   0.   0.25 0.   0.   0.25 0.   0.25 0.   0.   0.25 0.  ]
 [0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   

/usr/local/lib/python3.10/dist-packages/gymnasium/core.py:311: UserWarning: WARN: env.P to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.P` for environment variables or `env.get_wrapper_attr('P')` that will search the reminding wrappers.
  logger.warn(


## Exercise : Sample an episode

Sample an episode with the uniform policy and return the lists of observations (states), actions, rewards and the done flags.

Notice that the list of states should have one more element (the initial state).

Test the function by rendering an episode. We will use `RecordVideo` to do that.

In [ ]:
def sample_episode(env, policy, reset=True):
  # the functions takes the environment and policy adn sample
  #an episode of the environment according to the policy that has been given
  # it returns list of states, actions, rewards, dones at the episode sampled

    states = []
    actions = []
    rewards = []
    dones = []

    ### BEGIN SOLUTION
    # If reset, we reset the environment and get an initial state
    # else we set the initial state to it's current state env.env.s
    if reset:
        initial_state,_ = env.reset()
    else:
        initial_state = env.env.s
    done = False
    # Collect the initial state
    states.append(initial_state)
    # While the episode has not finished
    while not done:
        # Select an action
        action = policy.sample(states[-1])
        actions.append(action)
        # Step the environment
        obs, reward, terminated, truncated, info = env.step(action)
        # The episode is done if it has been terminated or truncated
        done = terminated or truncated
        # Collect the state, reward and action taken
        states.append(obs)
        rewards.append(reward)
        dones.append(done)
    ### END SOLUTION

    return states, actions, rewards, dones

env = RecordVideo(gym.make("FrozenLake-v1", render_mode="rgb_array"))
uniform_policy = UniformPolicy(env.action_space)
states, actions, rewards, dones = sample_episode(env, uniform_policy)
print(rewards)
print(len(states), len(actions), len(rewards), len(dones))
display(env.video())

<IPython.core.display.Javascript object>

[0.0, 0.0, 0.0, 0.0]
5 4 4 4


## Exercise : Returns of episode

For a sampled episode compute the return $G_t$ for all steps $t$.

**Hint** the return is easily computed backwards from the last step in the episode to the first.

In [ ]:
def compute_returns(rewards, gamma):
    '''
    computes return of an episode with discount factor gamma.
    returns list of returns.
    '''
    returns = np.zeros(len(rewards))

    ### BEGIN SOLUTION

    episode_len = len(rewards)
    returns[episode_len-1] = rewards[episode_len-1] #last return is always same as no future step

    # Iterate over the rewards backward computing each return
    for i,reward in enumerate(reversed(rewards)):
        if i==0:
            continue #as the last return is already computed
        returns[episode_len-1-i] = rewards[episode_len-1-i] + gamma*returns[episode_len-i]# using the previous return computed

    ### END SOLUTION
    return returns

env = gym.make("FrozenLake-v1", render_mode="rgb_array")
uniform_policy = UniformPolicy(env.action_space)

# Now we will run until one of them has a positive reward
while True:
    states, actions, rewards, dones = sample_episode(env, uniform_policy)
    if np.sum(rewards) > 0:
        # Print the rewards
        print(rewards)
        # Compute and print the returns
        gamma = 0.9
        returns = compute_returns(rewards, gamma)
        print(returns)
        # Exit the loop
        break

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]
[0.531441 0.59049  0.6561   0.729    0.81     0.9      1.      ]


# Policy evaluation

In this section we will implement various value function estimation methods:
- Direct method using the previous P and R
- Naive (cheating) method that assumes possibility of setting an arbitrary initial state
- Monte Carlo methods
- Time-difference method

## Exercise : Direct method

Solve the Bellman equation, using the direct solution (matrix inversion).

**Note** that this method leads to miss-leading values for terminal states, since there is no way to indicate terminality using the Bellman equation without sampling.

In [ ]:
def value_direct(env, gamma, policy):
    P, R, terminal_states = compute_P_and_R(env, policy)

    ### BEGIN SOLUTION
    #compute the value using bellman equation
    I = np.eye(env.observation_space.n)
    v = np.matmul(np.linalg.inv(I-gamma*P),R)

    ### END SOLUTION

    # Unsqueeze the extra dimension
    v = v[:,0]

    return v

env = gym.make("FrozenLake-v1", render_mode="rgb_array")
uniform_policy = UniformPolicy(env.action_space)
gamma = 0.9
v_direct = value_direct(env, gamma, uniform_policy)
print(v_direct)

[ 4.02953462e-02  3.80021094e-02  9.06008086e-02  3.70639671e-02  6.04976257e-02 -5.89215880e-16
  2.37003375e-01  0.00000000e+00  1.68085365e-01  5.18463074e-01  9.62747525e-01  0.00000000e+00
  5.00257691e-16  1.17344744e+00  3.52341144e+00  1.00000000e+01]


/usr/local/lib/python3.10/dist-packages/gymnasium/core.py:311: UserWarning: WARN: env.P to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.P` for environment variables or `env.get_wrapper_attr('P')` that will search the reminding wrappers.
  logger.warn(


***Solving the Bellaman equation: ひ = (I - γ P )^(-1) R***

## (Optional) Exercise : Verify the solution

Check if the computed value function $V$ fulfills the Bellman equation. You may use `np.allclose(a, b)` to check if all elements in `a` and `b` are close up to a numerical error.

In [ ]:
### BEGIN SOLUTION
P, R, terminal_states = compute_P_and_R(env, uniform_policy)
gamma = 0.9
v_direct = value_direct(env, gamma, uniform_policy)
v_original = R.squeeze() + gamma*np.matmul(P,v_direct)
np.allclose(v_direct,v_original)
### END SOLUTION

True

**the value function V satisfying the Bellman equation means that it correctly represents the total expected rewards for a state, considering both immediate rewards and future rewards.**

For the following we must set the value function of terminal states to 0 in order to keep our value function comparable to those computed by methods using episode sampling.

In [ ]:
# To compare to the following methods we must set the value of terminal states to 0
v_direct[terminal_states] = 0
print(v_direct)

[0.04029535 0.03800211 0.09060081 0.03706397 0.06049763 0.         0.23700338 0.         0.16808536
 0.51846307 0.96274753 0.         0.         1.17344744 3.52341144 0.        ]


***compare between ひ = R + γ P ひ and v_direct***

## Exercise : Naive (cheating) method

Compute the value function for each state by cheating (changing the starting state and computing the average return from the start).

In [ ]:
def value_naive(env, gamma, policy, n_episodes):
    '''
    This function returns the value function using naive method. Setting the initial state manually.
    '''
    ### BEGIN SOLUTION

    # Initialize values and counts tables (one cell per state)
    values = np.zeros((env.observation_space.n,1))
    counts = np.zeros((env.observation_space.n,1))


    # Compute the number of episodes per state
    episodes_per_state = int(n_episodes/env.observation_space.n)


    # For each initial state
    for i in range(env.observation_space.n):

        # For each episode
        for episode in range(episodes_per_state):
            # Reset the environment
            env.reset()

            # Set the initial state
            env.env.s = i

            # Sample an episode
            # (without reseting the environment to avoid changing the initial state)
            states, actions, rewards, dones = sample_episode(env, policy, reset = False)

            # Compute the returns
            returns = compute_returns(rewards, gamma)

            # Accumulate the return of the initial state
            values[i] += returns[0]

        # Divide the accumulated value by the number of episodes
        values[i] = values[i]/episodes_per_state

    ### END SOLUTION

    return values

def compute_value_error(v_est, v_ref):
    '''
    computes and returns the mean and standard deviation of the difference of two value function
    '''
    diff = (v_est - v_ref)
    return np.mean(diff), np.std(diff)


env = gym.make("FrozenLake-v1", render_mode=None)
uniform_policy = UniformPolicy(env.action_space)
gamma = 0.9
n_episodes = 10000
v_naive = value_naive(env, gamma, uniform_policy, n_episodes)
#print the naive value
print(v_naive)
#compute the mean and std of the error between the naive and direct method
print(compute_value_error(v_naive, v_direct))

[[0.00377907]
 [0.00472303]
 [0.00382959]
 [0.0062073 ]
 [0.00420658]
 [0.00308415]
 [0.004703  ]
 [0.00744366]
 [0.01315209]
 [0.00423213]
 [0.00436944]
 [0.0047248 ]
 [0.00415425]
 [0.00463438]
 [0.00370917]
 [0.00416003]]
(-0.42303158618941883, 0.8716214555879386)


***Here we compare between the direct value and the cheating value, by changing the starting state and computing the average return from the start and we find that the differnce between them is small . So, i think it's good for estimation.***

## Exercise : Monte Carlo first visit method
Compute the value function for each state using the Monte carlo method "first visit"

In [ ]:
def value_montecarlo_first(env, gamma, policy, n_episodes):
    '''
    returns the value function calculated using monte carlo first visit method
    '''
    ### BEGIN SOLUTION

    # Initialize values and counts tables (one cell per state)
    values = np.zeros((env.observation_space.n,1))
    counts = np.zeros((env.observation_space.n,1))

    # For each episode
    for episode in range(n_episodes):
        # Sample an episode and compute returns
        states, actions, rewards, dones = sample_episode(env, policy, reset = True)
        # print(states)
        # Keep track of visited states
        visited_states = set(states) #stores the states which has not been visited yet
        returns = compute_returns(rewards, gamma)

        # For each state and associated return
        for state, ret in zip(states, returns):
            # If first visit
            if state in visited_states:

                # Increment counts
                counts[state]+=1

                # Accumulate returns
                values[state]+=ret

                # Update the set of visited states
                visited_states.remove(state) #as state is visited remove it from the list

    # Average the accumulated returns
    for i in range(env.observation_space.n):
        if counts[i]!=0: #avoiding devide by 0
            values[i] = values[i]/counts[i]

    ### END SOLUTION

    return values

env = gym.make("FrozenLake-v1", render_mode=None)
uniform_policy = UniformPolicy(env.action_space)
gamma = 0.9
n_episodes = 10000
v_mc_firstvisit = value_montecarlo_first(env, gamma, uniform_policy, n_episodes)
print(v_mc_firstvisit)
print(compute_value_error(v_mc_firstvisit, v_direct))

[[0.00438747]
 [0.00416908]
 [0.00871994]
 [0.00237304]
 [0.00689208]
 [0.        ]
 [0.02282187]
 [0.        ]
 [0.01945457]
 [0.06319828]
 [0.10293562]
 [0.        ]
 [0.        ]
 [0.13363225]
 [0.41085346]
 [0.        ]]
(-0.37938627616373355, 0.8774907694692882)


***In montecarlo first visit, it offers good estimation for value function give a state. I think that the error will decrease if we sample more episodes.***

**from observation the mean error and standard error isn't low compared to the values and terminal states did not accumulate any values. In my opinion the results will be closer if we sample more episodes as monte carlo has a bias towards high variance values. as the proceses are randomized. The more episodes we  consider the more accurate the results will be.**



As we see here in th previous example I increased the number of episodes in order to checl wheter the error is not too high or not compared to the last run. and from observation the mean error and standard error is decresing compared to the values.

## Exercise : Monte Carlo every visit method
Compute the value function for each state using the Monte carlo method "every visit"

In [ ]:
def value_montecarlo_every(env, gamma, policy, n_episodes):
    ### BEGIN SOLUTION
    '''
    Calculates and returns the value function using the every-visit Monte Carlo method.'''
    # Initialize values and counts tables (one cell per state)
    values = np.zeros((env.observation_space.n,1))
    counts = np.zeros((env.observation_space.n,1))

    # For each episode
    for episode in range(n_episodes):
        # Sample an episode and compute returns
        states, actions, rewards, dones = sample_episode(env, policy, reset = True)
        returns = compute_returns(rewards, gamma)

        # For each state and associated return
        for state, ret in zip(states, returns):
            # Increment counts
            counts[state]+=1

            # Accumulate returns
            values[state]+=ret

    # Average the accumulated returns
    for i in range(env.observation_space.n):
        if counts[i]!=0:
            values[i] = values[i]/counts[i]


    ### END SOLUTION

    return values

env = gym.make("FrozenLake-v1", render_mode=None)
uniform_policy = UniformPolicy(env.action_space)
gamma = 0.9
n_episodes = 10000
v_mc_everyvisit = value_montecarlo_every(env, gamma, uniform_policy, n_episodes)
print(v_mc_everyvisit)
print(compute_value_error(v_mc_everyvisit, v_direct))

[[0.00467263]
 [0.00422787]
 [0.0113749 ]
 [0.00349721]
 [0.00706651]
 [0.        ]
 [0.02794363]
 [0.        ]
 [0.01727832]
 [0.05914662]
 [0.10123896]
 [0.        ]
 [0.        ]
 [0.1276557 ]
 [0.38224981]
 [0.        ]]
(-0.3814541196832151, 0.8767180301522679)


***Every visit shows less error than first visit because it considers all visited states so i think it is more computationaly effecient but it might provide overstimation of the value because it might visit the same state many times.***

## Exercise : Monte Carlo incremental

Implement the incremental Monte Carlo method.

In [ ]:
def value_montecarlo_incremental(env, gamma, policy, n_episodes):
    ### BEGIN SOLUTION
    # Initialize values and counts tables (one cell per state)
    values = np.zeros((env.observation_space.n,1))
    counts = np.zeros((env.observation_space.n,1))



    # For each episode
    for episode  in range(n_episodes):

        # Sample an episode and compute returns
        states, actions, rewards, dones  =sample_episode( env, policy, reset = True)
        returns = compute_returns(rewards, gamma)



        # For each state and associated return
        for state, ret in zip(states, returns):

            # Increment counts
            counts[state]+=1


            # Update value with return
            values[state] = values[state] + (ret-values[state])/counts[state]


    ### END SOLUTION
    return values

env = gym.make("FrozenLake-v1", render_mode=None)
uniform_policy = UniformPolicy(env.action_space)
gamma = 0.9
n_episodes = 10000
v_mc_incremental = value_montecarlo_incremental(env, gamma, uniform_policy, n_episodes)
print(v_mc_incremental)
print(compute_value_error(v_mc_incremental, v_direct))

[[0.00460375]
 [0.00378489]
 [0.00968342]
 [0.00350023]
 [0.00716025]
 [0.        ]
 [0.02433039]
 [0.        ]
 [0.02241067]
 [0.06150326]
 [0.10719106]
 [0.        ]
 [0.        ]
 [0.14956034]
 [0.38641339]
 [0.        ]]
(-0.3793422779863504, 0.8769871132709274)


***It shown that the error decreased much when using incremental method. it offers better estimate. this method uses the incremental mean it using incremental mean might introduce some bais.***



ひ(s ) = ひ(s ) + (Gt- ひ(s ))/N(s )

***Here each time we visit a state for the first time, we make some add of the reward to the total rewards for that state and then we divide the total reward by the number of times we have visited.***

## Exercise : Time-differences (TD) method

In [ ]:
def value_td(env, gamma, policy, n_episodes, alpha=0.4):
    ### BEGIN SOLUTION
    # Initialize value table (one cell per state)

    values  =np.zeros((env.observation_space.n,1))


    # For each episode
    for episode in range(n_episodes):

        # Sample an episode
        states, actions, rewards, dones = sample_episode(env, policy, reset = True)

        # Compute returns
        returns = compute_returns(rewards, gamma)


        # For each step in the episode
        for iter, state in enumerate(states):

            # Update the value of the depart state with the current value, the value of the next state and the reward
            if iter<len(rewards):
                values[state] = values[state] + alpha * (rewards[iter] + gamma *  values[states[iter + 1 ]] - values[state])



    ### END SOLUTION
    return values

env = gym.make("FrozenLake-v1", render_mode=None)
uniform_policy = UniformPolicy(env.action_space)
gamma = 0.9
n_episodes = 10000
alpha = 0.4
v_td = value_td(env, gamma, uniform_policy, n_episodes, alpha=alpha)
print(v_td)
print(compute_value_error(v_td, v_direct))

[[0.00255712]
 [0.00102516]
 [0.02832001]
 [0.00640434]
 [0.0071549 ]
 [0.        ]
 [0.03087741]
 [0.        ]
 [0.01844683]
 [0.01783088]
 [0.05307893]
 [0.        ]
 [0.        ]
 [0.20776446]
 [0.71983513]
 [0.        ]]
(-0.3597701816760055, 0.8890813937525508)


## (Optional) Exercise : Comparison TD to Monte Carlo incremental

How do these to methods compare? Explain the relative advantages and disadvantages.

## Exercise : Action-value
Compute the action value function using one or more of the following methods:
- Naive (cheating)
- MC "first visit"
- MC "every visit"
- MC incremental
- TD

In [ ]:
def actionvalue_montecarlo_every(env, gamma, policy, n_episodes):

    ### BEGIN SOLUTION

    q = np.zeros((env.observation_space.n,  env.action_space.n))
    counts = np.zeros((env.observation_space.n,  env.action_space.n))

    for episode in range(n_episodes):
        states, actions, rewards, dones = sample_episode(env, policy, reset = True)
        returns = compute_returns(rewards, gamma)

        for state, action, ret in zip(states, actions, returns):
            # Increment counts
            counts[state,action]+=1

            # Accumulate returns
            q[state, action]+=ret

    # Average the accumulated returns
    for i in range(env.observation_space.n): #for every state
        for j in range(env.action_space.n): #for every action in state
            if counts[i,j]!=0: #avoiding devide by 0
                q[i,j] = q[i,j]/counts[i,j]

    ### END SOLUTION
    return q

env = gym.make("FrozenLake-v1", render_mode="rgb_array")
uniform_policy = UniformPolicy(env.action_space)
gamma = 0.9
n_episodes = 10000
q_mc_everyvisit = actionvalue_montecarlo_every(env, gamma, uniform_policy, n_episodes)
print(q_mc_everyvisit)

[[0.00553102 0.0046945  0.00401654 0.00358794]
 [0.00119273 0.00556085 0.00311993 0.006122  ]
 [0.01506277 0.00878204 0.01349451 0.00648275]
 [0.0062117  0.00232981 0.00250184 0.01008888]
 [0.00818392 0.00785149 0.00728052 0.0028627 ]
 [0.         0.         0.         0.        ]
 [0.02366108 0.04274703 0.04246676 0.00369871]
 [0.         0.         0.         0.        ]
 [0.00706939 0.02006961 0.01991076 0.02449925]
 [0.03151318 0.08330988 0.08544894 0.0304367 ]
 [0.13492027 0.1793537  0.11649347 0.0391479 ]
 [0.         0.         0.         0.        ]
 [0.         0.         0.         0.        ]
 [0.04105227 0.13818256 0.11134491 0.18803416]
 [0.21184438 0.51013264 0.49538046 0.42469782]
 [0.         0.         0.         0.        ]]


# Policy improvement

In this section we are going to improve upon the uniform policy, which selects actions at random, independently on the state.

To assess whether our learned policies work, we will start by implmenting a scoring function to evaluate the policies.

## Exercise : Evaluate performance of a policy

In this environment we define a goal as obtaining a final reward greater than 0, thus reaching the target state, since it is the only one with a non-zero reward.

For a given policy, compute:
- the average number of episodes where the goal was reached
- the average number of steps to reach the goal


In [ ]:
def score_policy(env, gamma, policy, n_episodes):
    '''
    returns percentage of times the agent reached the goal and avg number of steps took to reach goal
    '''
    episodes_to_goal = 0
    steps_to_goal = []

    ### BEGIN SOLUTION
    for episode in range(n_episodes):
        states, actions, rewards, dones = sample_episode(env, policy, reset = True)
        #if the goal reached increase the number of times reached and store the number of steps required
        if rewards[-1]!=0:
            episodes_to_goal+=1
            steps_to_goal.append(len(rewards))
    ### END SOLUTION

    return episodes_to_goal/n_episodes, np.mean(steps_to_goal)


In [ ]:
score_policy(env, 0.9, uniform_policy, 10000)

(0.0123, 12.504065040650406)

## Exercise : Greedy policy (policy improvement)

Create a greedy policy from an action value function and evaluate it's performance.

The greedy policy selects the action leading to the largest value in the current state.

**Note** that when several actions have the same maximal value, it is best to randomly pick one of them.

Pick the first highest value action or (optionally) pick randomly amongst actions with the highest value.

In [ ]:
class GreedyPolicy(UniformPolicy):
    def __init__(self, action_space, q):
        super().__init__(action_space) #initialize the parent class it is inherting from
        # q is the action-value table
        self.q = q

    def _max_value_action(self, state):
        ### BEGIN SOLUTION
        action = np.argmax(self.q[state]) #index of max value in the state

        ### END SOLUTION
        return action

    def probability(self, state, action):
        # Select the highest value action
        action_max = self._max_value_action(state)
        # Return a probability of 1 for the selected action 0 otherwise
        action_prob = float(action == action_max)

        return action_prob

    def sample(self, state):
        # Select the highest value action
        action_max = self._max_value_action(state)

        return action_max

env = gym.make("FrozenLake-v1", render_mode=None)
greedy_policy = GreedyPolicy(env.action_space, q_mc_everyvisit) #intializing with the q value computed using mc every visit

## Exercise : Compare the performance of policies

Report the performance of the uniform policy and the greedy policy

In [ ]:
env = gym.make("FrozenLake-v1", render_mode=None)

### BEGIN SOLUTION

un_per,un_mean =  score_policy(env, 0.9, uniform_policy, 10000)
print(f"uniform policy: {un_per:.2%} / {un_mean}")

gr_per,gr_mean = score_policy(env, 0.9, greedy_policy, 10000)
print(f"greedy policy: {gr_per:.2%} / {gr_mean}")

### END SOLUTION

uniform policy: 1.21% / 13.84297520661157
greedy policy: 18.03% / 23.427066001109264


***The result is as expected as the uniform policy is just taking random action whatever the state is. But the greedy policy is taking the action that will give maximum return in every state from knowledge of the q table***

In [ ]:
env = RecordVideo(gym.make("FrozenLake-v1", render_mode="rgb_array", is_slippery = True))
for i in range(10):
    states, actions, rewards, dones = sample_episode(env, greedy_policy)
display(env.video())

<IPython.core.display.Javascript object>

# Policy learning

In this section we will start learning policies.

We will then implement the following policy learning methods:
- policy iteration
- SARSA
- Q-Learning

## Exercise : Policy iteration
By alternating between policy evaluation and policy improvement, find an optimal policy.

Print the scores of each intermediate policy and comment on how the metrics evolve.

In [ ]:
def policy_learn_iteration(env, initial_policy, policy_evaluation_function,
                           n_episodes_value, n_episodes_score, n_iterations):
    policy = initial_policy #initializing policy with the given one

    ### BEGIN SOLUTION
    # Score and print the initial policy
    per,mean = score_policy(env, 0.9, policy, n_episodes_score)
    print(f"Initial Policy: {per:.2%} / {mean}")
    action_value_prev = None #at the start nothing is learned
    # Policy evaluation
    for it in range(n_iterations):
        if it==0:
            per,mean = score_policy(env, 0.9, policy, n_episodes_score)
            print(f"Policy (iter: {it}): {per:.2%} / {mean:.4}, Value change: nan")


        # Keep track of the action-value function change
        # (sum of absolute difference between the previous
        #  and next action-value funcitons)
        # Note that at the first step we will report a change of nan
        # since we still don't have a previous action-value function
        action_value = policy_evaluation_function(env, gamma, policy, n_episodes_value) #using initial policy getting the q table
        if it!=0:
            diff = np.abs(np.diff(action_value_prev - action_value)) #CHANGE OCCURED IN THE Q TABLE

        action_value_prev = action_value #storing the q table

        # Policy improvement
        policy = GreedyPolicy(env.action_space, action_value) #initializing greedy policy to learn

        # Policy scoring
        if it!=0: #score the greedy policy to see how much it is learning

            per,mean = score_policy(env, 0.9, policy, n_episodes_score)
            print(f"Policy (iter: {it}): {per:.2%} / {mean:.4}, Value change: {np.sum(diff):.3}")





    ### END SOLUTION

env = gym.make("FrozenLake-v1", render_mode=None)
uniform_policy = UniformPolicy(env.action_space)

policy_learn_iteration(env, uniform_policy, actionvalue_montecarlo_every,
                       n_episodes_value=1000, n_episodes_score=10000,
                       n_iterations=30)

Initial Policy: 1.38% / 12.68840579710145
Policy (iter: 0): 1.22% / 12.61, Value change: nan
Policy (iter: 1): 3.91% / 11.58, Value change: 1.97
Policy (iter: 2): 4.34% / 11.95, Value change: 0.406
Policy (iter: 3): 4.25% / 11.54, Value change: 0.23
Policy (iter: 4): 4.37% / 11.8, Value change: 0.241
Policy (iter: 5): 4.17% / 11.66, Value change: 0.263
Policy (iter: 6): 4.15% / 11.1, Value change: 0.176
Policy (iter: 7): 4.17% / 11.37, Value change: 0.103
Policy (iter: 8): 3.78% / 11.26, Value change: 0.256
Policy (iter: 9): 4.05% / 11.25, Value change: 0.157
Policy (iter: 10): 4.14% / 11.12, Value change: 0.276
Policy (iter: 11): 4.02% / 11.16, Value change: 0.56
Policy (iter: 12): 3.80% / 11.24, Value change: 0.334
Policy (iter: 13): 3.92% / 11.4, Value change: 0.209
Policy (iter: 14): 3.68% / 10.9, Value change: 0.401
Policy (iter: 15): 3.71% / 11.3, Value change: 0.491
Policy (iter: 16): 3.39% / 11.22, Value change: 0.704
Policy (iter: 17): 3.77% / 11.81, Value change: 0.306
Policy

## Exercise : Epsilon-greedy policy

One of the issues with the greedy policy is that some possible trajectories may never be visited depending on the initial estimation of the value function.

To avoid this create an epsilon-greedy policy. These policies act differently when running in training mode and evaluation mode. In evaluation mode they act as a greedy policy. In training mode:
- with probability epsilon, uniformly selects an action
- else selects the action with maximum value (greedy policy)

Implement the `EpsilonGreedyPolicy` class.


In [ ]:
class EpsilonGreedyPolicy(GreedyPolicy):
    def __init__(self, action_space, q, epsilon,
                 epsilon_decay=1, epsilon_min=0):
        super().__init__(action_space, q)
        self.epsilon_start = epsilon
        self.epsilon_decay = epsilon_decay
        self.epsilon_min = epsilon_min

        self.epsilon = self.epsilon_start

    def sample(self, state):
        ### BEGIN SOLUTION
        #if in training mode explore epsilon percentage of time else exploit the knowlegde it has
        if np.random.random()<= self.epsilon and self.training:
            choices = [i for i in range(self.n_actions)]
            action = np.random.choice(choices)
        else:
            action = np.argmax(self.q[state])
        ### END SOLUTION
        return action

    def begin_episode(self, episode_index):
        # Start of an episode
        self.epsilon = self.epsilon_start * (self.epsilon_decay ** episode_index)
        self.epsilon = max(self.epsilon, self.epsilon_min)

# Instantiate a policy
dummy = EpsilonGreedyPolicy(env.action_space, q_mc_everyvisit, epsilon=0.5)

# Sample 20 actions from state 0 in train mode
dummy.train()
actions = [dummy.sample(0) for i in range(20)]
print(actions)

# Sample 20 actions from state 0 in eval mode
dummy.eval()
actions = [dummy.sample(0) for i in range(20)]
print(actions)

[3, 0, 1, 1, 2, 3, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


## Exercise : SARSA

State–action–reward–state–action (SARSA) is a method to learn a policy leveraging the TD estimation of action-value functions and an epsilon-greedy policy.

Since it is based on TD, it does not require full episodes to train, the policy can be improved at each step. We will therefore **not use the `sample_episode`** and reimplement here a similar loop.

Implement the SARSA algorithm, in this case it will be passed a policy of type `EpsilonGreedyPolicy` which stores the action-value (or Q-table). We may access and modify if using `policy.q`.

Score the trained policy.

In [ ]:
def policy_learn_sarsa(env, policy, gamma, n_episodes, alpha, max_n_steps=1000, print_every=500):
    # No need to create the tables of action-values since they are stored directly in the policy

    # While we haven't reached the desired number of steps
    for episode in range(n_episodes):
        # Call the policy begin_episode so it can handle epsilon decay
        policy.begin_episode(episode)

        # Print every couple episodes
        if not episode % print_every:
            print(f'ep: {episode}, epsilon: {policy.epsilon:.3f}')
            policy.eval() #go into evaluate mode
            avg_episodes_to_goal,avg_steps_to_goal = score_policy(env, gamma, policy, 10000)
            print(f'Policy SARSA: {avg_episodes_to_goal:.2%} / {avg_steps_to_goal:.2f}')
            policy.train() #go back to training
            print()

        # Get initial state
        state, info = env.reset()

        # Take the first action acording to the policy
        action = policy.sample(state)

        # While we haven't reached the maximum number of steps for the episode
        for step in range(max_n_steps):
            ### BEGIN SOLUTION
            # Perform a step of the environment

            state_next,reward,terminated,truncated,info =  env.step(action)

            # It is done if terminated or truncated
            done = terminated or truncated


            # If episode has finished
            if done:
                # Update the action-value table and leave the loop
                action_next = policy.sample(state_next)
                policy.q[state, action] += alpha * (reward + gamma * policy.q[state_next, action_next] - policy.q[state, action])
                break



            # Sample the next action
            action_next = policy.sample(state_next)

            # Update the action-value table
            policy.q[state, action] += alpha * (reward + gamma * policy.q[state_next, action_next] - policy.q[state, action])
            ### END SOLUTION

            # Set the current state and action
            state = state_next
            action = action_next

    return

env = gym.make("FrozenLake-v1", render_mode=None)

q_initial = np.zeros((env.observation_space.n, env.action_space.n))
sarsa_policy = EpsilonGreedyPolicy(env.action_space, q_initial,
                                   epsilon=1, epsilon_decay=0.999, epsilon_min=0.01)

gamma = 0.9
n_episodes = 5000
alpha = 0.2
n_episodes_score = 1000

### BEGIN SOLUTION
# Train the policy
sarsa_policy.train()
policy_learn_sarsa(env, sarsa_policy, gamma, n_episodes, alpha, max_n_steps=1000, print_every=500)

### END SOLUTION

### BEGIN SOLUTION
# Evaluate the policy
sarsa_policy.eval()
avg_episodes_to_goal,avg_steps_to_goal = score_policy(env, gamma, sarsa_policy, n_episodes_score)

### END SOLUTION

print(f'Policy SARSA: {avg_episodes_to_goal:.2%} / {avg_steps_to_goal:.2f}')

ep: 0, epsilon: 1.000


/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Policy SARSA: 0.00% / nan

ep: 500, epsilon: 0.606
Policy SARSA: 16.09% / 18.92

ep: 1000, epsilon: 0.368
Policy SARSA: 9.02% / 15.48

ep: 1500, epsilon: 0.223
Policy SARSA: 11.27% / 24.57

ep: 2000, epsilon: 0.135
Policy SARSA: 39.61% / 33.54

ep: 2500, epsilon: 0.082
Policy SARSA: 38.97% / 27.55

ep: 3000, epsilon: 0.050
Policy SARSA: 51.34% / 31.97

ep: 3500, epsilon: 0.030
Policy SARSA: 26.23% / 23.03

ep: 4000, epsilon: 0.018
Policy SARSA: 50.49% / 32.99

ep: 4500, epsilon: 0.011
Policy SARSA: 37.39% / 29.85

Policy SARSA: 48.90% / 33.26


***We can see SARSA gives much better result in terms of percentage of time the agent reaches the goal compared to the normal policy learning. Also because here we are using the eps greedy policy. The agent will try to reach every state. But for greedy policy the agent might never reach some state as it doesnot explore***

## Exercise : Q-learning

Q-learning is very similar to SARSA. **SARSA is an on-policy learning method** in which the action-values are updated following the same policy. In SARSA the action-values are updated using the value of the next state and next action taken.

Q-learning is off-policy, it does not assume the same policy when updating the action-values, instead it assumes an optimal policy by using the maximum value of the next state.

Implement a modified version of `policy_learn_sarsa` that performs Q-learning.

In [ ]:
def policy_learn_qlearn(env, policy, gamma, n_episodes, alpha, max_n_steps=1000, print_every=500):
    # No need to create the tables of action-values since they are stored directly in the policy

    # While we haven't reached the desired number of steps
    for episode in range(n_episodes):
        # Call the policy begin_episode so it can handle epsilon decay
        policy.begin_episode(episode)

        # Print every couple episodes
        if not episode % print_every:
            print(f'ep: {episode}, epsilon: {policy.epsilon:.3f}')

        # Get initial state
        state, info = env.reset()

        # Take the first action acording to the policy
        action = policy.sample(state)

        # While we haven't reached the maximum number of steps for the episode
        for step in range(max_n_steps):
            ### BEGIN SOLUTION
            # Perform a step of the environment
            state_next,reward,terminated,truncated,info =  env.step(action)

            # Set as done if terminated or truncated
            done = terminated or truncated


            # If episode has finished
            if done:
                # Update the action-value table and leave the loop
                policy.q[state, action] += alpha*(reward + gamma*np.max(policy.q[state_next]) - policy.q[state,action])
                break



            # Sample the next action
            action_next = policy.sample(state_next)


            # Update the action-value table
            policy.q[state, action] += alpha*(reward + gamma*np.max(policy.q[state_next]) - policy.q[state,action])

            ### END SOLUTION

            # Set the current state and action
            state = state_next
            action = action_next

    return

env = gym.make("FrozenLake-v1", render_mode=None)

q_initial = np.zeros((env.observation_space.n, env.action_space.n))
qlearn_policy = EpsilonGreedyPolicy(env.action_space, q_initial,
                                    epsilon=1, epsilon_decay=0.999, epsilon_min=0.01)

gamma = 0.9
n_episodes = 5000
alpha = 0.2
n_episodes_score = 1000

### BEGIN SOLUTION
# Train the policy
qlearn_policy.train()
policy_learn_qlearn(env, qlearn_policy, gamma, n_episodes, alpha, max_n_steps=1000, print_every=500)


### END SOLUTION

### BEGIN SOLUTION
# Evaluate the policy
qlearn_policy.eval()
avg_episodes_to_goal,avg_steps_to_goal = score_policy(env, gamma, qlearn_policy, n_episodes_score)


### END SOLUTION

print(f'Policy Q-learn: {avg_episodes_to_goal:.2%} / {avg_steps_to_goal:.2f}')


ep: 0, epsilon: 1.000
ep: 500, epsilon: 0.606
ep: 1000, epsilon: 0.368
ep: 1500, epsilon: 0.223
ep: 2000, epsilon: 0.135
ep: 2500, epsilon: 0.082
ep: 3000, epsilon: 0.050
ep: 3500, epsilon: 0.030
ep: 4000, epsilon: 0.018
ep: 4500, epsilon: 0.011
Policy Q-learn: 73.40% / 37.12


## (Optional) Exercise : Train for Taxi

Train an epsilon-greedy policy using Q-learning on the `Taxi-v3` environment.

Score the performance of this policy and compare it to a uniform policy.

In [ ]:
### BEGIN SOLUTION



env = gym.make("Taxi-v3", render_mode=None)

q_initial = np.zeros((env.observation_space.n, env.action_space.n))
qlearn_policy = EpsilonGreedyPolicy(env.action_space, q_initial,
                                    epsilon=1, epsilon_decay=0.999, epsilon_min=0.01)

gamma = 0.9
n_episodes = 5000
alpha = 0.2
n_episodes_score = 1000

# Train the policy
qlearn_policy.train()
policy_learn_qlearn(env, qlearn_policy, gamma, n_episodes, alpha, max_n_steps=1000, print_every=500)


# Evaluate the policy
qlearn_policy.eval()
avg_episodes_to_goal,avg_steps_to_goal = score_policy(env, gamma, qlearn_policy, n_episodes_score)

print(f'Policy Q-learn: {avg_episodes_to_goal:.2%} / {avg_steps_to_goal:.2f}')


### END SOLUTION

ep: 0, epsilon: 1.000
ep: 500, epsilon: 0.606
ep: 1000, epsilon: 0.368
ep: 1500, epsilon: 0.223
ep: 2000, epsilon: 0.135
ep: 2500, epsilon: 0.082
ep: 3000, epsilon: 0.050
ep: 3500, epsilon: 0.030
ep: 4000, epsilon: 0.018
ep: 4500, epsilon: 0.011
Policy Q-learn: 100.00% / 13.06


***here we find that the Q-learn policy is better (100%) because Taxi is a deterministic environment, so it has an excellent poilcy here.***

## (Optional) Exercise : Render Taxi

Run 3 episodes of a Q-learn trained policy on `Taxi-v3` this time rendering the result.

In [ ]:
### BEGIN SOLUTION

env = gym.make("Taxi-v3", render_mode=None)
env = RecordVideo(gym.make("Taxi-v3", render_mode="rgb_array"))
qlearn_policy.eval()
env.reset()
for i in range(3):
    env.render()
    states, actions, rewards, dones = sample_episode(env, qlearn_policy)

env.close()

### END SOLUTION

<IPython.core.display.Javascript object>

## (Bonus) Exercise : A observation space environment

Try to perform Q-learning on an environment (e.g. `CartPole-v1`) with continuous action and observation spaces.

You will need to discretize the observation space.

In [ ]:
# Example of rendering and showing a CartPole-v1 environment
env = RecordVideo(gym.make("CartPole-v1", render_mode="rgb_array"))
observation, info = env.reset()
i=0
while True:
    env.render()

    action = env.action_space.sample()
    state_next, reward, terminated, truncated, info = env.step(action)
    done = terminated or truncated

    if done:
      break

env.close()

<IPython.core.display.Javascript object>

In [ ]:
import numpy as np

class ObservationWrapper(gym.ObservationWrapper):
    '''
    n_states(int): number of state of the env.
                It has to be the some power of N = env.observation_space.shape[0]
    bounds(list of tuples): bounds of the states of the env.(taken from the gymnasium documentation).
                each tuple gives the bounds of the states
    '''
    def __init__(self, env, n_states=4096, bounds=[(-4.8,4.8),(-50,50),( -0.418, 0.418),(-20,20)]):
        super().__init__(env)

        self.n_states = n_states
        self.bounds = bounds
        assert isinstance(env.observation_space, gym.spaces.box.Box)

        # Get the dimensions of the observation space
        self.dimension = env.observation_space.shape[0]
        self.states_per_dimention = int(n_states**(1/self.dimension)) #we give equal number of state per dimension

        # calculating steps for each state and storing in a list
        self.step_list = []
        for state_bound in bounds:
            step = (state_bound[1] - state_bound[0])/self.states_per_dimention
            self.step_list.append(step)


    def get_quant_observation(self, new_obs):
        ##calculate the new quantized state index from a list of states.  (like converting number systems)
        #observations are the digits and states per dimention is the base of the number system.
        quantized_obs = 0
        dimension = self.dimension
        for i, obs in enumerate(new_obs):
            quantized_obs+= obs*self.states_per_dimention**(dimension-i-1)

        # print(quantized_obs)
        return int(quantized_obs)

    def observation(self, obs):
        ### BEGIN SOLUTION
        # Quantize the observations (states)
        new_obs = []
        # get the quanta of each state
        for i,observation_state in enumerate(obs):
            new_obs.append(int((observation_state-self.bounds[i][0])/self.step_list[i]))

        # Once quantized each dimension compute a single observation index
        return self.get_quant_observation(new_obs)




In [ ]:
def score_policy(env, gamma, policy, n_episodes):
    #score policy for cart-pole environment as the scoring is different
    #Here as the score we are taking how many steps the agent survived.
    #the longer it survived the better the policy
    episodes_to_goal = 0
    steps_to_goal = []

    for episode in range(n_episodes):
        states, actions, rewards, dones = sample_episode(env, policy, reset = True)
        episodes_to_goal+= len(states)

    return episodes_to_goal/n_episodes,steps_to_goal

In [ ]:
# Initialize environment, wrap to render
env = gym.make("CartPole-v1", render_mode="rgb_array")

# Wrap to discretize the observation space
env = ObservationWrapper(env)

### BEGIN SOLUTION
# Initialize policy

observation, info = env.reset()
#action is 2 and states are 4096 after quantization
q_initial = np.zeros((4096, 2))
qlearn_policy = EpsilonGreedyPolicy(env.action_space, q_initial,
                                    epsilon=1, epsilon_decay=0.999, epsilon_min=0.01)

gamma = 0.9
n_episodes = 5000
alpha = 0.1
n_episodes_score = 100

### BEGIN SOLUTION
# Train the policy
qlearn_policy.train()
policy_learn_qlearn(env, qlearn_policy, gamma, n_episodes, alpha, max_n_steps=1000, print_every=500)

### END SOLUTION

### BEGIN SOLUTION
# Evaluate the policy
qlearn_policy.eval()
avg_episodes_to_goal = score_policy(env, gamma, qlearn_policy, n_episodes_score)


### END SOLUTION

print(f'Policy Q-learn: {avg_episodes_to_goal}')



ep: 0, epsilon: 1.000
ep: 500, epsilon: 0.606
ep: 1000, epsilon: 0.368
ep: 1500, epsilon: 0.223
ep: 2000, epsilon: 0.135
ep: 2500, epsilon: 0.082
ep: 3000, epsilon: 0.050
ep: 3500, epsilon: 0.030
ep: 4000, epsilon: 0.018
ep: 4500, epsilon: 0.011
Policy Q-learn: (466.66, [])


In [ ]:
qlearn_policy.eval()
avg_episodes_to_goal = score_policy(env, gamma, qlearn_policy, n_episodes_score)
print(f'Policy Q-learn: {avg_episodes_to_goal}')


Policy Q-learn: (450.54, [])


In [ ]:
env = gym.make("CartPole-v1", render_mode="rgb_array")
# Wrap to discretize the observation space and then wrap with the record video to render vdo
env = RecordVideo(ObservationWrapper(env))
env.reset()
qlearn_policy.eval()
for i in range(1):
    env.render()
    state_next, reward, terminated, truncated = sample_episode(env, qlearn_policy)

env.close()

<IPython.core.display.Javascript object>